In [1]:
import numpy as np
import pandas as pd
import os
import openpyxl
import tensorflow as tf
from openpyxl import load_workbook
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from pandas import DataFrame
from sklearn.preprocessing import MinMaxScaler
from pandas import concat
from tensorflow.keras.layers import LSTM,Dense,GRU,Dropout,SimpleRNN
from sklearn.metrics import mean_squared_error
# from keras.callbacks import EarlyStopping, ModelCheckpoint

In [2]:
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j + 1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j + 1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j + 1, i)) for j in range(n_vars)]
    # put it all together
    agg = concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

In [3]:
n_hours = 3
n_features = 43
train_num_24 = 2789
train_num_25 = 2892
train_num_19 = 1934

In [4]:
from os.path import join
df = pd.read_excel('rain_data.xlsx')
df.head()

,홍수사상번호,연,월,일,시간,유입량,데이터집단 1,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 38,Unnamed: 39,Unnamed: 40,데이터 집단 6,Unnamed: 42,Unnamed: 43,Unnamed: 44,Unnamed: 45,Unnamed: 46,Unnamed: 47
0,1,2006,7,10,8,189.100000,6.4,7,7,7,...,8,2.54,122.660,6.4,7,7,8,8,2.54,122.610
1,1,2006,7,10,9,216.951962,6.3,7,8,7,...,10,2.53,122.648,7.3,7,8,10,10,2.53,122.600
2,1,2006,7,10,10,251.424419,6.4,7,9,7,...,11,2.53,122.636,8.2,7,9,10,11,2.53,122.590
3,1,2006,7,10,11,302.812199,7.3,7,10,7,...,14,2.53,122.620,11.3,9,10,15,14,2.53,122.585
4,1,2006,7,10,12,384.783406,8.2,7,12,8,...,16,2.53,122.604,14.4,12,12,18,16,2.53,122.575


In [5]:
# df_metrix 만드는 과정
df_matrix = df.to_numpy()
df_matrix = df_matrix[0:train_num_25, 5:]
print(df_matrix)

df_matrix = df_matrix.astype('float32')

scaler = MinMaxScaler(feature_range=(0, 1))
df_matrix = scaler.fit_transform(df_matrix)

df_matrix

[[189.1         6.4         7.        ...   8.          2.54
  122.61     ]
 [216.9519618   6.3         7.        ...  10.          2.53
  122.6      ]
 [251.4244187   6.4         7.        ...  11.          2.53
  122.59     ]
 ...
 [481.1030831   3.3854      1.        ...   1.          2.96
  137.165    ]
 [470.4581089   3.2841      1.        ...   1.          2.94
  137.18     ]
 [        nan  14.2576     32.        ...   0.          1.93
  120.515    ]]


array([[0.0086303 , 0.01948843, 0.0212766 , ..., 0.03065134, 0.09392971,
        0.15531301],
       [0.00992569, 0.01918392, 0.0212766 , ..., 0.03831417, 0.09329073,
        0.15491581],
       [0.01152899, 0.01948843, 0.0212766 , ..., 0.04214559, 0.09329073,
        0.1545186 ],
       ...,
       [0.02221129, 0.01030877, 0.00303951, ..., 0.00383142, 0.12076677,
        0.7334657 ],
       [0.0217162 , 0.0100003 , 0.00303951, ..., 0.00383142, 0.11948882,
        0.73406124],
       [       nan, 0.04341535, 0.09726444, ..., 0.        , 0.05495208,
        0.07209539]], dtype=float32)

In [6]:
# print(df_matrix)
lag = series_to_supervised(df_matrix, n_hours,1)

lag = lag.values

tmp = lag.shape[1]-42

lag = lag[:,0:tmp]
print(lag.shape)

(2888, 130)


In [7]:
n_train_hours = train_num_19
train = lag[:n_train_hours, :]
test = lag[n_train_hours:, :]

In [8]:
##1
n_obs = n_hours * n_features
train_X, train_y = train[:, :n_obs], train[:, -n_features]
test_X, test_y = test[:, :n_obs], test[:, -n_features]
print(train_X.shape, len(train_X), train_y.shape)
# reshape input to be 3D [samples, timesteps, features]
train_X = train_X.reshape((train_X.shape[0], n_hours, n_features))
test_X = test_X.reshape((test_X.shape[0], n_hours, n_features))
print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)

(1934, 129) 1934 (1934,)
(1934, 3, 43) (1934,) (954, 3, 43) (954,)


In [9]:
# #testing
# model = tf.keras.Sequential()
# model.add(GRU(units = 50,
#               # dropout =0.2,
#               return_sequences = True,
#               input_shape = (train_X.shape[1],train_X.shape[2]),
#               activation = 'tanh'))

# model.add(SimpleRNN(50))

# model.add(Dense(1))

In [10]:
# testing
model = tf.keras.Sequential()
# model.add(GRU(units = 50,
#               # dropout =0.2,
#               return_sequences = True,
#               input_shape = (train_X.shape[1],train_X.shape[2]),
#               activation = 'tanh'))

# model.add(SimpleRNN(50))
model.add(LSTM(20, input_shape = (train_X.shape[1],train_X.shape[2]), activation = 'tanh', return_sequences = False))
model.add(Dense(1))

2021-08-26 17:21:12.610210: I tensorflow/core/platform/cpu_feature_guard.cc:143] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2021-08-26 17:21:12.624880: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f968b397b90 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2021-08-26 17:21:12.624894: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version


In [ ]:
# # optimizer =tf.train.AdamOptimizer(lea)
adam = tf.keras.optimizers.Adam(lr =0.01)
# early_stop = EarlyStopping(monitor='val_loss', patience=5)
# filename = os.path.join(model_path, 'tmp_checkpoint.h5')
# checkpoint = ModelCheckpoint(filename, monitor='val_loss', verbose=1, save_best_only=True, mode='auto')
adam = tf.keras.optimizers.Adam(lr =0.0001)
model.compile(loss='mse', optimizer='adam')

# history = model.fit(train_X, train_y, 
#                                     epochs=100, 
#                                     batch_size=16,
#                                     validation_data=(test_X, test_y), 
#                                     callbacks=[early_stop, checkpoint])
history = model.fit(train_X, train_y, 
                                    epochs=5000, 
                                    batch_size=32,
                                    validation_data=(test_X, test_y))

pred = model.predict(test_X)

Epoch 1/5000
61/61 [==============================] - 1s 10ms/step - loss: 0.0073 - val_loss: 7.7619e-04
Epoch 2/5000
61/61 [==============================] - 0s 4ms/step - loss: 0.0013 - val_loss: 4.7649e-04
Epoch 3/5000
61/61 [==============================] - 0s 3ms/step - loss: 6.9297e-04 - val_loss: 4.2580e-04
Epoch 4/5000
61/61 [==============================] - 0s 3ms/step - loss: 4.5906e-04 - val_loss: 2.7789e-04
Epoch 5/5000
61/61 [==============================] - 0s 4ms/step - loss: 3.6205e-04 - val_loss: 2.4713e-04
Epoch 6/5000
61/61 [==============================] - 0s 3ms/step - loss: 3.0120e-04 - val_loss: 1.9200e-04
Epoch 7/5000
61/61 [==============================] - 0s 3ms/step - loss: 2.7215e-04 - val_loss: 2.2227e-04
Epoch 8/5000
61/61 [==============================] - 0s 3ms/step - loss: 2.1865e-04 - val_loss: 1.5977e-04
Epoch 9/5000
61/61 [==============================] - 0s 3ms/step - loss: 1.7690e-04 - val_loss: 1.4353e-04
Epoch 10/5000
61/61 [==============

61/61 [==============================] - 0s 3ms/step - loss: 2.1433e-05 - val_loss: 1.3445e-05
Epoch 77/5000
61/61 [==============================] - 0s 3ms/step - loss: 1.8878e-05 - val_loss: 1.3429e-05
Epoch 78/5000
61/61 [==============================] - 0s 3ms/step - loss: 1.5967e-05 - val_loss: 2.1101e-05
Epoch 79/5000
61/61 [==============================] - 0s 3ms/step - loss: 1.5614e-05 - val_loss: 2.0411e-05
Epoch 80/5000
61/61 [==============================] - 0s 3ms/step - loss: 1.6899e-05 - val_loss: 1.3379e-05
Epoch 81/5000
61/61 [==============================] - 0s 3ms/step - loss: 1.7109e-05 - val_loss: 1.2763e-05
Epoch 82/5000
61/61 [==============================] - 0s 3ms/step - loss: 1.6226e-05 - val_loss: 1.5364e-05
Epoch 83/5000
61/61 [==============================] - 0s 3ms/step - loss: 1.9927e-05 - val_loss: 2.1526e-05
Epoch 84/5000
61/61 [==============================] - 0s 3ms/step - loss: 1.4955e-05 - val_loss: 1.7054e-05
Epoch 85/5000
61/61 [============

Epoch 151/5000
61/61 [==============================] - 0s 3ms/step - loss: 9.3779e-06 - val_loss: 1.1046e-05
Epoch 152/5000
61/61 [==============================] - 0s 3ms/step - loss: 1.9784e-05 - val_loss: 1.6071e-05
Epoch 153/5000
61/61 [==============================] - 0s 3ms/step - loss: 9.3527e-06 - val_loss: 9.6147e-06
Epoch 154/5000
61/61 [==============================] - 0s 3ms/step - loss: 6.7050e-06 - val_loss: 8.8503e-06
Epoch 155/5000
61/61 [==============================] - 0s 3ms/step - loss: 6.1694e-06 - val_loss: 7.2569e-06
Epoch 156/5000
61/61 [==============================] - 0s 3ms/step - loss: 8.7270e-06 - val_loss: 9.2637e-06
Epoch 157/5000
61/61 [==============================] - 0s 4ms/step - loss: 7.3181e-06 - val_loss: 1.4242e-05
Epoch 158/5000
61/61 [==============================] - 0s 3ms/step - loss: 1.4097e-05 - val_loss: 1.3637e-05
Epoch 159/5000
61/61 [==============================] - 0s 3ms/step - loss: 1.0506e-05 - val_loss: 7.1129e-06
Epoch 160/

61/61 [==============================] - 0s 3ms/step - loss: 1.1091e-05 - val_loss: 6.4880e-06
Epoch 226/5000
61/61 [==============================] - 0s 3ms/step - loss: 1.7821e-05 - val_loss: 2.8920e-05
Epoch 227/5000
61/61 [==============================] - 0s 3ms/step - loss: 9.1201e-06 - val_loss: 5.7607e-06
Epoch 228/5000
61/61 [==============================] - 0s 3ms/step - loss: 5.8494e-06 - val_loss: 7.6500e-06
Epoch 229/5000
61/61 [==============================] - 0s 3ms/step - loss: 3.3705e-06 - val_loss: 6.3631e-06
Epoch 230/5000
61/61 [==============================] - 0s 3ms/step - loss: 4.3713e-06 - val_loss: 5.0801e-06
Epoch 231/5000
61/61 [==============================] - 0s 3ms/step - loss: 2.9270e-06 - val_loss: 7.3853e-06
Epoch 232/5000
61/61 [==============================] - 0s 3ms/step - loss: 4.8024e-06 - val_loss: 6.2849e-06
Epoch 233/5000
61/61 [==============================] - 0s 3ms/step - loss: 4.8904e-06 - val_loss: 6.4570e-06
Epoch 234/5000
61/61 [===

61/61 [==============================] - 0s 3ms/step - loss: 1.8077e-06 - val_loss: 3.6952e-06
Epoch 300/5000
61/61 [==============================] - 0s 3ms/step - loss: 2.8706e-06 - val_loss: 5.6048e-06
Epoch 301/5000
61/61 [==============================] - 0s 3ms/step - loss: 2.7737e-06 - val_loss: 4.5127e-06
Epoch 302/5000
61/61 [==============================] - 0s 3ms/step - loss: 3.1098e-06 - val_loss: 4.1229e-06
Epoch 303/5000
61/61 [==============================] - 0s 3ms/step - loss: 2.1689e-06 - val_loss: 3.8440e-06
Epoch 304/5000
61/61 [==============================] - 0s 3ms/step - loss: 2.6312e-06 - val_loss: 4.3379e-06
Epoch 305/5000
61/61 [==============================] - 0s 3ms/step - loss: 1.5546e-06 - val_loss: 3.6479e-06
Epoch 306/5000
61/61 [==============================] - 0s 3ms/step - loss: 1.7875e-06 - val_loss: 3.2887e-06
Epoch 307/5000
61/61 [==============================] - 0s 3ms/step - loss: 3.1233e-06 - val_loss: 3.6579e-06
Epoch 308/5000
61/61 [===

...

Epoch 4443/5000
61/61 [==============================] - 0s 2ms/step - loss: 9.7319e-08 - val_loss: 1.0858e-06
Epoch 4444/5000
61/61 [==============================] - 0s 2ms/step - loss: 5.8844e-08 - val_loss: 9.8159e-07
Epoch 4445/5000
61/61 [==============================] - 0s 2ms/step - loss: 7.9950e-08 - val_loss: 9.7882e-07
Epoch 4446/5000
61/61 [==============================] - 0s 2ms/step - loss: 5.6608e-08 - val_loss: 9.6019e-07
Epoch 4447/5000
61/61 [==============================] - 0s 2ms/step - loss: 8.1838e-08 - val_loss: 1.2779e-06
Epoch 4448/5000
61/61 [==============================] - 0s 2ms/step - loss: 4.1008e-07 - val_loss: 1.1234e-06
Epoch 4449/5000
61/61 [==============================] - 0s 2ms/step - loss: 1.8864e-07 - val_loss: 1.0946e-06
Epoch 4450/5000
61/61 [==============================] - 0s 2ms/step - loss: 4.0325e-07 - val_loss: 8.6526e-07
Epoch 4451/5000
61/61 [==============================] - 0s 3ms/step - loss: 1.6166e-07 - val_loss: 1.2672e-06
E

Epoch 4517/5000
61/61 [==============================] - 0s 2ms/step - loss: 5.4512e-08 - val_loss: 1.0237e-06
Epoch 4518/5000
61/61 [==============================] - 0s 2ms/step - loss: 8.4303e-08 - val_loss: 1.1599e-06
Epoch 4519/5000
61/61 [==============================] - 0s 2ms/step - loss: 1.0321e-07 - val_loss: 1.1567e-06
Epoch 4520/5000
61/61 [==============================] - 0s 2ms/step - loss: 1.2036e-07 - val_loss: 1.0252e-06
Epoch 4521/5000
61/61 [==============================] - 0s 2ms/step - loss: 6.6852e-08 - val_loss: 9.7772e-07
Epoch 4522/5000
61/61 [==============================] - 0s 2ms/step - loss: 1.0873e-06 - val_loss: 4.1212e-06
Epoch 4523/5000
61/61 [==============================] - 0s 3ms/step - loss: 4.2407e-06 - val_loss: 1.1375e-06
Epoch 4524/5000
61/61 [==============================] - 0s 2ms/step - loss: 3.9286e-07 - val_loss: 1.2186e-06
Epoch 4525/5000
61/61 [==============================] - 0s 2ms/step - loss: 1.5473e-07 - val_loss: 1.0223e-06
E

Epoch 4591/5000
61/61 [==============================] - 0s 3ms/step - loss: 2.8489e-06 - val_loss: 1.4114e-06
Epoch 4592/5000
61/61 [==============================] - 0s 2ms/step - loss: 3.5211e-07 - val_loss: 1.1364e-06
Epoch 4593/5000
61/61 [==============================] - 0s 2ms/step - loss: 1.9179e-07 - val_loss: 1.0408e-06
Epoch 4594/5000
61/61 [==============================] - 0s 2ms/step - loss: 5.3414e-08 - val_loss: 1.0333e-06
Epoch 4595/5000
61/61 [==============================] - 0s 3ms/step - loss: 8.3112e-08 - val_loss: 1.0205e-06
Epoch 4596/5000
61/61 [==============================] - 0s 2ms/step - loss: 7.3556e-08 - val_loss: 1.0194e-06
Epoch 4597/5000
61/61 [==============================] - 0s 2ms/step - loss: 4.3037e-07 - val_loss: 1.5732e-06
Epoch 4598/5000
61/61 [==============================] - 0s 2ms/step - loss: 3.6053e-07 - val_loss: 9.7636e-07
Epoch 4599/5000
61/61 [==============================] - 0s 2ms/step - loss: 2.0287e-07 - val_loss: 9.6347e-07
E

Epoch 4665/5000
61/61 [==============================] - 0s 2ms/step - loss: 2.5740e-07 - val_loss: 1.0464e-06
Epoch 4666/5000
61/61 [==============================] - 0s 3ms/step - loss: 1.2911e-07 - val_loss: 1.2732e-06
Epoch 4667/5000
61/61 [==============================] - 0s 3ms/step - loss: 9.6693e-08 - val_loss: 9.3833e-07
Epoch 4668/5000
61/61 [==============================] - 0s 2ms/step - loss: 1.5783e-07 - val_loss: 8.9931e-07
Epoch 4669/5000
61/61 [==============================] - 0s 2ms/step - loss: 1.0987e-07 - val_loss: 1.0161e-06
Epoch 4670/5000
61/61 [==============================] - 0s 2ms/step - loss: 2.4135e-07 - val_loss: 1.2963e-06
Epoch 4671/5000
61/61 [==============================] - 0s 2ms/step - loss: 6.6426e-07 - val_loss: 1.8333e-06
Epoch 4672/5000
61/61 [==============================] - 0s 2ms/step - loss: 3.0232e-06 - val_loss: 4.4363e-06
Epoch 4673/5000
61/61 [==============================] - 0s 2ms/step - loss: 2.6001e-06 - val_loss: 1.2756e-06
E

Epoch 4739/5000
61/61 [==============================] - 0s 3ms/step - loss: 1.5825e-07 - val_loss: 9.5564e-07
Epoch 4740/5000
61/61 [==============================] - 0s 2ms/step - loss: 7.3897e-08 - val_loss: 1.0210e-06
Epoch 4741/5000
61/61 [==============================] - 0s 2ms/step - loss: 4.6149e-08 - val_loss: 9.7014e-07
Epoch 4742/5000
61/61 [==============================] - 0s 2ms/step - loss: 3.4489e-07 - val_loss: 1.3934e-06
Epoch 4743/5000
61/61 [==============================] - 0s 2ms/step - loss: 5.9758e-07 - val_loss: 1.3982e-06
Epoch 4744/5000
61/61 [==============================] - 0s 2ms/step - loss: 1.7129e-07 - val_loss: 9.1887e-07
Epoch 4745/5000
61/61 [==============================] - 0s 2ms/step - loss: 1.8148e-07 - val_loss: 9.4704e-07
Epoch 4746/5000
61/61 [==============================] - 0s 2ms/step - loss: 1.7330e-07 - val_loss: 1.0053e-06
Epoch 4747/5000
61/61 [==============================] - 0s 2ms/step - loss: 1.0908e-07 - val_loss: 9.2840e-07
E

In [ ]:
RMSE = mean_squared_error(test_y,pred)**0.5
print(RMSE)

In [ ]:
plt.figure(figsize=(12, 9))
plt.plot(history.history['loss'],label = 'train')
plt.plot(history.history['val_loss'], label = 'test')

plt.show()

In [ ]:
plt.figure(figsize=(12, 9))
plt.plot(pred,label='prediction')
plt.plot(test_y,label='real')
plt.show()